In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

np.set_printoptions(precision=4)

In [ ]:
data = pd.read_csv('final_df.csv')

In [ ]:
data.tail()

In [ ]:
data['season'].unique()

In [ ]:
df = data.copy()

train = df[df.season<2023]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [ ]:
def score_regression(model):
    score = 0
    for circuit in df[df.season == 2023]['round'].unique():

        test = df[(df.season == 2023) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2023]['round'].unique().max()
    return model_score

In [ ]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

In [ ]:
from sklearn import svm
import joblib
import numpy as np

params = {'gamma': np.logspace(-4, -1, 10),
          'C': np.logspace(-2, 1, 10),
          'kernel': ['rbf']} 


best_score = -np.inf 
best_model = None
best_params = None

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = svm.SVR(gamma=gamma, C=c, kernel=kernel)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('svm_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)
            
            if model_score > best_score:
                best_score = model_score
                best_model = model
                best_params = model_params

# Save the best model
if best_model:
    best_model_filename = f"best_svm_model_gamma_{best_params[0]}_C_{best_params[1]}_kernel_{best_params[2]}.pkl"
    joblib.dump(best_model, best_model_filename)
    print(f"Best model saved as {best_model_filename} with score {best_score}")
